In [12]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from typing import Dict, List

def make_cointegration_tests(
    data: pd.DataFrame,
    columns_a: List[str],
    columns_b: List[str],
    alpha: float = 0.05,
    verbose: bool = False
) -> Dict:
    """
    Perform pairwise Engle-Granger cointegration tests between time series.

    Parameters:
        data: DataFrame of time series.
        columns_a: List of column names (set A).
        columns_b: List of column names (set B).
        alpha: Significance level for detecting cointegration.
        verbose: If True, print intermediate results.

    Returns:
        Dictionary with:
            - 'columns_a': columns_a
            - 'columns_b': columns_b
            - 'results': pairwise result dictionary
            - 'cointegration_matrix': binary matrix with detection flags
    """
    pairs = [[a, b] for a in columns_a for b in columns_b if a != b]
    n = len(columns_a)
    m = len(columns_b)

    matrix_index = sorted(set(columns_a).union(columns_b))
    cointegration_matrix = pd.DataFrame(0, index=matrix_index, columns=matrix_index)

    results = {'column_a': columns_a, 'columns_b': columns_b, 'results': {}}

    for a, b in pairs:
        series_a = data[a].dropna()
        series_b = data[b].dropna()

        # Align by index
        aligned_df = pd.concat([series_a, series_b], axis=1).dropna()
        y1 = aligned_df.iloc[:, 0]
        y2 = aligned_df.iloc[:, 1]

        score, p_value, _ = coint(y1, y2)

        detected = p_value < alpha
        results['results'][f"{a}__{b}"] = {
            'column_a': a,
            'column_b': b,
            'p_value': p_value,
            'detection': detected
        }

        if verbose:
            print(f"Test {a} <-> {b}: p-value = {p_value:.4f}, detected = {detected}")

        if detected:
            cointegration_matrix.loc[a, b] = 1
            cointegration_matrix.loc[b, a] = 1  # symmetric

    results['cointegration_matrix'] = cointegration_matrix
    return results


In [13]:

def create_cointeg_matrix_from_dict(cointeg_dict):
    d = cointeg_dict

    if len(d['column_a']) < 1:
        raise Exception()

    # Specific case: 1 x n Dataframe
    if len(d['column_a']) == 1:
        df_columns, df_values = [], []
        df_index = d['column_a']
        rd = d['results']
        for key in rd.keys():
            df_columns.append(rd[key]['column_b'])
            df_values.append(rd[key]['detection'])
        res_df = pd.DataFrame([df_values], columns=df_columns, index=[df_index])
        return res_df

    # General case: m x n Dataframe
    df_columns, df_values = [], []
    df_index = d['column_a']
    df_columns = d['columns_b']
    res_df = pd.DataFrame(columns=df_columns, index=df_index)
    rd = d['results']
    for key in rd.keys():
        res_df.loc[rd[key]['column_a'], rd[key]['column_b']] = rd[key]['detection']
    mask = np.triu(np.ones(res_df.shape), k=0).astype(bool)
    # df_masked = res_df.mask(mask)
    df_masked = res_df.where(~mask, other='')
    return df_masked



# Create example data
import numpy as np
np.random.seed(0)

n = 200
x = np.cumsum(np.random.randn(n))  # non-stationary
y = x + np.random.normal(scale=0.5, size=n)  # cointegrated with x
z = np.cumsum(np.random.randn(n))  # independent random walk

df = pd.DataFrame({
    'x': x,
    'y': y,
    'z': z
})

# Run cointegration tests
cointegration_results = make_cointegration_tests(df, columns_a=['x', 'y', 'z'], columns_b=['x', 'y', 'z'], verbose=True)

# print("\nCointegration Matrix:")
# print(cointegration_results['cointegration_matrix'])

detection_table = create_cointeg_matrix_from_dict(cointegration_results)
detection_table


Test x <-> y: p-value = 0.0000, detected = True
Test x <-> z: p-value = 0.7822, detected = False
Test y <-> x: p-value = 0.0000, detected = True
Test y <-> z: p-value = 0.7327, detected = False
Test z <-> x: p-value = 0.9793, detected = False
Test z <-> y: p-value = 0.9695, detected = False


,x,y,z
x,,,
y,True,,
z,False,False,


🧠 **Insight**

- Cointegration implies a shared long-term stochastic trend.
- Even if short-run Granger causality is absent, cointegration suggests long-term equilibrium constraints.
- Use it to filter out truly "tied" series in macroeconomic datasets or pairs trading.


Would you like this expanded to support Johansen testing for groups (e.g., 3+ series), or visual tools to show cointegrated residuals over time?

Great question — and a very important one when interpreting **cointegration tests**, particularly the **Engle-Granger test**.

---

### 🔍 Does the order of series matter in Engle-Granger?

**Yes, it does.**
The Engle-Granger two-step test **is not symmetric** — that is, testing `x → y` is **not** guaranteed to give the same result as testing `y → x`.

#### Why?

Because the Engle-Granger test works as follows:

1. It **regresses `y` on `x`**:

   $$
   y_t = \beta x_t + \varepsilon_t
   $$
2. Then it tests the **residuals `ε_t`** from this regression for stationarity.

If you switch the roles of `x` and `y`, you’re changing:

* Which variable is the **dependent** one (being explained)
* The direction of the **error correction mechanism**

So:

* `coint(x, y)` ≠ `coint(y, x)` in general.

---

### 💡 Interpretation Insight

In the Engle-Granger framework:

* The dependent variable (e.g., `y`) is assumed to be "adjusting" toward the long-run equilibrium with the independent one (`x`).
* If you're modeling economic relationships (e.g., prices in different markets), think carefully about **which variable is expected to respond** in the long run.

---

### ✅ What To Do in Practice?

If you're using Engle-Granger in **exploratory pairwise mode** (like your case), a common practice is:

* Test **both directions**, i.e., test `x → y` and `y → x`.
* Flag cointegration **if either direction** reports a significant result.

```python
score1, pval1, _ = coint(x, y)
score2, pval2, _ = coint(y, x)

detected = (pval1 < alpha) or (pval2 < alpha)
```

This gives you a **symmetrized detection rule**, useful for matrix-style outputs.

---

### ✳️ Bonus: Johansen Test is Symmetric

If you move to the **Johansen test** (for ≥2 series), it:

* Considers the system as a whole.
* Is **invariant to ordering** of the series.

---

Would you like a version of the `make_cointegration_tests` function that tests **both directions** for each pair?
